# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
   

/Users/nataliam/Documents/PY/Project_2_Data_Modeling_Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# Prints out how many files were processed: 
print(len(full_data_rows_list))

#uncomment the code below if you would like to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
# Number of rows in the final csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


### Data file
Generated CSV file:
![image_event_datafile_new.jpg](images/image_event_datafile_new.jpg)

#### Creating a Cluster

In [5]:
# Importing Cluster module to connect to Cassandra instance  locally (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()

# Establishing connection to be able to execute queries:
session = cluster.connect()

#### Create Keyspace

In [6]:
# Query to set up Keyspace: 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

#### Set Keyspace

In [7]:
# Setting KEYSPACE to 'sparkify'
session.set_keyspace('sparkify')

### Modeling  database tables  on the below type of queries:

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
![music_app_history](images/music_app_history.png)

#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
![user_songs_in_session](images/user_songs_in_session.png)

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
![users_listened_this_song](images/users_listened_this_song.png)

In [8]:
## Creating table 'music_app_history' to perform Query 1:
# (Give me the artist, song title and song's length in the music app history that was heard during sessionId = '', and itemInSession = '')
# PRIMARY KEY designed on two columns:  sessionId  and itemInSession 
# which will retrieve unique combination of session and item in the session in accordance with requirement

query = "CREATE TABLE IF NOT EXISTS music_app_history"
query = query + """(sessionId INT, 
                 itemInSession INT, 
                 artist_name TEXT, 
                 song_title TEXT, 
                 length float, 
                 PRIMARY KEY ((sessionId, itemInSession)))"""

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
# Reading CSV data file:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Populating data with INSERT statement into music_app_history         
        query = "INSERT INTO music_app_history (sessionId, itemInSession, artist_name, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))) # each column references to the column in CSV file
            
            

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# Performing SELECT query to get expected data from music_app_history:
rows = session.execute('SELECT * FROM music_app_history WHERE sessionId=338 AND itemInSession=4')
for row in rows:
    print (row[0], row[1], row[2], row[3], row[4])


338 4 Faithless 495.30731201171875 Music Matters (Mark Knight Dub)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
# Creating table 'user_songs_in_session' to collect data, required for Query 2:
#(retrieve name of artist, song (sorted by itemInSession) and user (first, last name) for userid = '', sessionid = '')
# COMPOSITE KEY consists of :
### PRIMARY KEY (sessionId, userId) which allows to get requested unique combination of userId and sessionId.
### CLUSTERING KEY itemInSession which is used to sort songs list sorted in accordance with requirenment

query = "CREATE TABLE IF NOT EXISTS user_songs_in_session"
query = query + """(sessionId INT, 
                 userId INT, 
                 itemInSession INT, 
                 user_fname TEXT, 
                 user_lname TEXT, 
                 artist_name TEXT, 
                 song_title TEXT, 
                 PRIMARY KEY((sessionId, userId),itemInSession))"""

try:
    session.execute(query)
except Exception as e:
    print(e)           

In [12]:
# Reading CSV data file and performing ETL to populate data into the table:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        
#INSERT query to populate data into 'user_songs_in_session' table :
        query = "INSERT INTO user_songs_in_session (sessionId, userId, itemInSession, user_fname, user_lname, artist_name, song_title)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[1], line[4], line[0], line[9]))
            

In [13]:
# Performing Query 2 with SELCT statement:
rows = session.execute('SELECT * FROM user_songs_in_session WHERE userid = 10 AND sessionid = 182')
for row in rows:
    print (row[0], row[1], row[2], row[3], row[4], row[5], row[6])
    

182 10 0 Down To The Bone Keep On Keepin' On Sylvie Cruz
182 10 1 Three Drives Greece 2000 Sylvie Cruz
182 10 2 Sebastien Tellier Kilometer Sylvie Cruz
182 10 3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
# Creating table users_listened_this_song for Query 3: 
#(Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own')
# COMPOSITE key consists of: 
### PRIMARY KEY song_title / to find song by title as provided in requirement
### CLUSTERING KEY userId / to retrieve all users who listened for a selected song
query = "CREATE TABLE IF NOT EXISTS users_listened_this_song"
query = query + """(song_title TEXT, 
                 userId INT, 
                 user_fname TEXT, 
                 user_lname TEXT,  
                 PRIMARY KEY((song_title), userId))"""
try:
    session.execute(query)
except Exception as e:
    print(e)
             

In [15]:
# Reading data CSV file:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:

#INSERT query to populate data into users_listened_this_song table
        query = "INSERT INTO users_listened_this_song (song_title, userId, user_fname, user_lname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
# Performing Query 3 with SELET statement: 
rows = session.execute("SELECT user_fname, user_lname FROM users_listened_this_song WHERE song_title ='All Hands Against His Own'")
for row in rows:
    print (row[0], row[1])
    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
# Drop all tables before closing out the session
query = "DROP TABLE music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE user_songs_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE users_listened_this_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()